In [1]:
%pip install requests
%pip install bs4
%pip install xlwt
%pip install selenium
%pip install tqdm
%pip install progress

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [34]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import time
import random

import re
import json
import requests

from datetime import datetime

from tqdm import tqdm
from time import sleep
from progress.bar import Bar

In [35]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

## Web Scraping- One2Car

In [36]:
data_lst = []
key_lst = []
err_count = 0
page_update = 1
page_number = 1710 #Start page # Next start 501, 1001, 1501,
page_number_end = 1774 #End page last page at 1774
#page_range = list(range(1,6))
page_range = (page_number_end-page_number)+1
pbar = tqdm(total=page_range, desc="Loading", bar_format="{l_bar}{bar} [ time left: {remaining} ]")


while True:
    # Update the progress bar
    pbar.update(1)
    time.sleep(0.1)

    try :
        urls = "https://www.one2car.com/%E0%B8%A3%E0%B8%96-%E0%B8%AA%E0%B8%B3%E0%B8%AB%E0%B8%A3%E0%B8%B1%E0%B8%9A-%E0%B8%82%E0%B8%B2%E0%B8%A2?page_number={}&page_size=26".format(page_number)
        response = get(urls, headers=headers)
        if response.status_code == 200:
            pass
            #print(f"Page{page_number}: Connect with URL Successful")
        elif response.status_code == 404:
            print(f"Page{page_number}: Error 404 page not found")
            continue
        else:
            print(f"Page{page_number}: Not both 200 and 404")
            continue

        html_soup = BeautifulSoup(response.text, 'html.parser')

        # Get .json data process
        content_list = html_soup.find_all('script')[2].text.strip()
        middle = len(content_list)//2
        under = content_list.rfind("itemListElement", 0, middle)
        content_list1 = html_soup.find_all('script')[2].text.strip()[under:]
        over = content_list1.find("[",0, middle)
        content_list2 = html_soup.find_all('script')[2].text.strip()[under+over:-2]
        #print(content_list)
        #print(content_list1)

        data_js = json.loads(content_list2)
        # print(data_js)

        df = pd.json_normalize(data_js)
        #print(df.head(10))

        data_lst.append(df)
        key_lst.append('Page'+str(page_number))
        
        #Write Data
        if (page_number % 10 == 0) | (page_number == page_number_end):
            data_result = pd.concat(data_lst, keys=key_lst)
            #set filename 
            now = datetime.now() # current date and time
            date_time = now.strftime("%Y%m%d_%H%M%S")
            #print("date and time:",date_time)
            #print(f'EndPage {page_number} Data Number: {len(data_result)}\n')
            # CSV file
            data_result.to_csv(f"{date_time}_P{page_number}_one2car_data_noclean.csv", encoding="utf-8-sig")
            # JSON file
            data_result.to_json(f"{date_time}_P{page_number}_one2car_data_noclean.json", orient='records')
            data_lst,key_lst = [],[]

        if page_number == page_number_end:
            print(f">>>>>>>>>>> End Process at Page no.{page_number_end} <<<<<<<<<<<<")
            break
        page_number+=1
        page_update+=1
        
    except ValueError:
        print(">>>>>>>>>>> Process Error <<<<<<<<<<<<")
        continue
        err_count+=1
        if err_count== 5:
            break
pbar.close()
print("Task completed!")


Loading:   0%|           [ time left: ? ]

Loading: 100%|██████████ [ time left: 00:00 ]

>>>>>>>>>>> End Process at Page no.1774 <<<<<<<<<<<<
Task completed!


##### Test Read JSON file

In [14]:
import json

# Specify the path to your JSON file
file_path = "01_DATA_One2Car/01_SomeColumn_Wrong_NoClean/20230621_133403_one2car_data_noclean.json"

try:
    # Open the file in read mode
    with open(file_path, 'r') as file:
        # Load the JSON data
        data = json.load(file)
        
        # Access and work with the data as needed
        print(data)  # Example: Print the JSON data
        
        # Close the file (not necessary with the 'with' statement)
        file.close()

except FileNotFoundError:
    print("File not found or path is incorrect.")

except json.JSONDecodeError:
    print("Invalid JSON format.")

df = pd.json_normalize(data)
df.head(10)

[{'@type': 'ListItem', 'position': 1, 'item.@type': 'Product', 'item.additionalType': 'Car', 'item.name': '2015 Toyota Corolla Altis 1.8 (ปี 14-18) E Sedan', 'item.image': ['https://img2.icarcdn.com/7729669/thumb-l_used-car-one2car-toyota-corolla-altis-e-sedan-thailand_c9dc99ab16206b49114e2c929c697ac0_jcGca4MFjpUCK8VjfXsiuJ_000007729669_b1c775d5_939b_4718_ae39_512b8f6a019b.JPG.webp?smia=xTM'], 'item.description': '-ลูกค้ามีเครดิต ผ่อนบ้านผ่อนรถ ออกรถได้ทุกอาชีพ รู้ผลภายใน1วัน ...', 'item.brand.@type': 'Brand', 'item.brand.name': 'Toyota', 'item.author.@type': 'Person', 'item.author.name': 'One2Car', 'item.model': 'Corolla Altis', 'item.bodyType': 'Sedan', 'item.fuelType': 'Petrol - Unleaded (ULP)', 'item.seatingCapacity': 5, 'item.color': 'สีดำ', 'item.mainEntityOfPage': 'https://www.one2car.com/for-sale/toyota-corolla-altis-e-กรุงเทพและปริมณฑล-กาญจนาภิเษก/9669277?auc=true', 'item.url': 'https://www.one2car.com/%E0%B8%A3%E0%B8%96-%E0%B8%AA%E0%B8%B3%E0%B8%AB%E0%B8%A3%E0%B8%B1%E0%B8%9A-%

#### Test

In [242]:
base_url = "https://www.one2car.com/%E0%B8%A3%E0%B8%96-%E0%B8%AA%E0%B8%B3%E0%B8%AB%E0%B8%A3%E0%B8%B1%E0%B8%9A-%E0%B8%82%E0%B8%B2%E0%B8%A2?page_number=1&page_size=26"
response = get(base_url, headers=headers)
if response.status_code == 200:
    print("Connect with URL Successful")
elif response.status_code == 404:
    print("Error 404 page not found")
else:
    print("Not both 200 and 404")


Connect with URL Successful


In [243]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [258]:
content_list = html_soup.find_all('script')[2].text.strip()
print(content_list)

[{
"@context": "http://schema.org",
"@type": ["ItemList", "Car"],
"additionalType": "Product",
"name": "2019 Nissan March 1.2 (ปี 10-21) E Hatchback",
"brand": {
"@type": "Thing",
"name": "Nissan"},
"model": "March",
"itemListOrder": "http://schema.org/ItemListOrderAscending",
"numberOfItems": 26,
"itemListElement": [

{

    "@type": "ListItem",
    "position": 1,
    "item": {"@type":"Product","additionalType":"Car","name":"2019 Nissan March 1.2 (ปี 10-21) E Hatchback","image":["https://img3.icarcdn.com/1016499/thumb-l_%E0%B8%A3%E0%B8%96%E0%B8%A1%E0%B8%B7%E0%B8%AD%E0%B8%AA%E0%B8%AD%E0%B8%87-one2car-nissan-march-e-hatchback-thailand_c9dc99ab16206b49114e8d4204b56fb4_q8PzO5nBVygZWV4BGpfRRf_000001016499_34d23f9b_6b38_4499_b637_0daae5fa4ff2.jpg.webp?smia=xTM"],"description":"🔥ดอกเบี้ยเริ่ม2.69% nissan march 1.2 e 2019 a/t ผ่อนเริ่ม 5,2xx ผ่อนสบาย ...","brand":{"@type":"Brand","name":"Nissan"},"author":{"@type":"Person","name":"One2Car"},"model":"March","bodyType":"Hatchback","fuelType":"P

In [274]:
middle = len(content_list)//2
under = content_list.rfind("itemListElement", 0, middle)
content_list1 = html_soup.find_all('script')[2].text.strip()[under:]
print(content_list1)


itemListElement": [

{

    "@type": "ListItem",
    "position": 1,
    "item": {"@type":"Product","additionalType":"Car","name":"2019 Nissan March 1.2 (ปี 10-21) E Hatchback","image":["https://img3.icarcdn.com/1016499/thumb-l_%E0%B8%A3%E0%B8%96%E0%B8%A1%E0%B8%B7%E0%B8%AD%E0%B8%AA%E0%B8%AD%E0%B8%87-one2car-nissan-march-e-hatchback-thailand_c9dc99ab16206b49114e8d4204b56fb4_q8PzO5nBVygZWV4BGpfRRf_000001016499_34d23f9b_6b38_4499_b637_0daae5fa4ff2.jpg.webp?smia=xTM"],"description":"🔥ดอกเบี้ยเริ่ม2.69% nissan march 1.2 e 2019 a/t ผ่อนเริ่ม 5,2xx ผ่อนสบาย ...","brand":{"@type":"Brand","name":"Nissan"},"author":{"@type":"Person","name":"One2Car"},"model":"March","bodyType":"Hatchback","fuelType":"Petrol - Unleaded (ULP)","seatingCapacity":5,"color":"สีขาว","mainEntityOfPage":"https://www.one2car.com/for-sale/nissan-march-e-กรุงเทพและปริมณฑล-มีนบุรี/9946101?auc=true","url":"https://www.one2car.com/%E0%B8%A3%E0%B8%96-%E0%B8%AA%E0%B8%B3%E0%B8%AB%E0%B8%A3%E0%B8%B1%E0%B8%9A-%E0%B8%82%E0%B8%B2%E0%B

In [276]:
over = content_list1.find("[",0, middle)
content_list2 = html_soup.find_all('script')[2].text.strip()[under+over:-2]
print(content_list2)

[

{

    "@type": "ListItem",
    "position": 1,
    "item": {"@type":"Product","additionalType":"Car","name":"2019 Nissan March 1.2 (ปี 10-21) E Hatchback","image":["https://img3.icarcdn.com/1016499/thumb-l_%E0%B8%A3%E0%B8%96%E0%B8%A1%E0%B8%B7%E0%B8%AD%E0%B8%AA%E0%B8%AD%E0%B8%87-one2car-nissan-march-e-hatchback-thailand_c9dc99ab16206b49114e8d4204b56fb4_q8PzO5nBVygZWV4BGpfRRf_000001016499_34d23f9b_6b38_4499_b637_0daae5fa4ff2.jpg.webp?smia=xTM"],"description":"🔥ดอกเบี้ยเริ่ม2.69% nissan march 1.2 e 2019 a/t ผ่อนเริ่ม 5,2xx ผ่อนสบาย ...","brand":{"@type":"Brand","name":"Nissan"},"author":{"@type":"Person","name":"One2Car"},"model":"March","bodyType":"Hatchback","fuelType":"Petrol - Unleaded (ULP)","seatingCapacity":5,"color":"สีขาว","mainEntityOfPage":"https://www.one2car.com/for-sale/nissan-march-e-กรุงเทพและปริมณฑล-มีนบุรี/9946101?auc=true","url":"https://www.one2car.com/%E0%B8%A3%E0%B8%96-%E0%B8%AA%E0%B8%B3%E0%B8%AB%E0%B8%A3%E0%B8%B1%E0%B8%9A-%E0%B8%82%E0%B8%B2%E0%B8%A2?page_number=1

In [277]:
data_js = json.loads(content_list2)
data_js

In [279]:
df = pd.json_normalize(data_js)
df.head(10)

In [2]:
df['item.description'][0]

NameError: name 'df' is not defined

In [280]:
df

,@type,position,item.@type,item.additionalType,item.name,item.image,item.description,item.brand.@type,item.brand.name,item.author.@type,...,item.offers.price,item.offers.priceCurrency,item.offers.itemCondition,item.offers.availability,item.offers.seller.@type,item.offers.seller.homeLocation.@type,item.offers.seller.homeLocation.address.@type,item.offers.seller.homeLocation.address.addressLocality,item.offers.seller.homeLocation.address.addressRegion,item.offers.seller.homeLocation.address.url
0,ListItem,1,Product,Car,2019 Nissan March 1.2 (ปี 10-21) E Hatchback,[https://img3.icarcdn.com/1016499/thumb-l_%E0%...,🔥ดอกเบี้ยเริ่ม2.69% nissan march 1.2 e 2019 a/...,Brand,Nissan,Person,...,314000,THB,https://schema.org/UsedCondition,https://schema.org/InStock,Person,Place,PostalAddress,กรุงเทพมหานคร,กรุงเทพและปริมณฑล,https://www.one2car.com/dealer/BoyAutogroup
1,ListItem,2,Product,Car,2018 Honda City 1.5 (ปี 14-18) V+ i-VTEC Sedan AT,[https://img1.icarcdn.com/86465121/thumb-l_%E0...,สภาพยางยังดีอยู่ สภาพยอดเยี่ยม เลขไมล์ต่ำ,Brand,Honda,Person,...,450000,THB,https://schema.org/UsedCondition,https://schema.org/InStock,Person,Place,PostalAddress,สระบุรี,ภาคกลาง,NaN
2,ListItem,3,Product,Car,2022 Ford Ranger 2.2 OPEN CAB (ปี 15-21) XL Pi...,[http://img.youtube.com/vi/xsgq5ZwzbBQ/maxresd...,🔥b1049 ford ranger open cab 2.2 xl plus (mnc) ...,Brand,Ford,Person,...,444000,THB,https://schema.org/UsedCondition,https://schema.org/InStock,Person,Place,PostalAddress,กรุงเทพมหานคร,กรุงเทพและปริมณฑล,https://www.one2car.com/dealer/BoyAutogroup
3,ListItem,4,Product,Car,2019 Ford Ranger 2.2 DOUBLE CAB (ปี 15-21) Hi-...,[https://img2.icarcdn.com/1886889/thumb-l_%E0%...,🔥ดอกเบี้ยเริ่ม2.59% ford ranger double cab hi-...,Brand,Ford,Person,...,624000,THB,https://schema.org/UsedCondition,https://schema.org/InStock,Person,Place,PostalAddress,กรุงเทพมหานคร,กรุงเทพและปริมณฑล,https://www.one2car.com/dealer/BoyAutogroup
4,ListItem,5,Product,Car,2019 Mazda 3 2.0 (ปี 14-18) 2.0 S Sports Hatch...,[https://img1.icarcdn.com/27335121/thumb-l_%E0...,สภาพยอดเยี่ยม เพิ่งทำการเช็คสภาพรถ มีประวัติกา...,Brand,Mazda,Person,...,419999,THB,https://schema.org/UsedCondition,https://schema.org/InStock,Person,Place,PostalAddress,กรุงเทพมหานคร,กรุงเทพและปริมณฑล,NaN
5,ListItem,6,Product,Car,2018 Mercedes-Benz GLA250 2.0 W156 (ปี 14-20) ...,[https://img5.icarcdn.com/79688311/thumb-l_%E0...,2018 mercedes-benz gla250 2.0 w156 (ปี 14-20) ...,Brand,Mercedes-Benz,Person,...,1499000,THB,https://schema.org/UsedCondition,https://schema.org/InStock,Person,Place,PostalAddress,กรุงเทพมหานคร,กรุงเทพและปริมณฑล,https://www.one2car.com/dealer/f6e70e2c-d84a-4...
6,ListItem,7,Product,Car,2019 Mitsubishi Pajero Sport 2.4 (ปี 19-24) GT...,[https://img1.icarcdn.com/48529911/thumb-l_%E0...,🔥a1866 mitsubishi pajero 2.4 gt premium 2019 a...,Brand,Mitsubishi,Person,...,939000,THB,https://schema.org/UsedCondition,https://schema.org/InStock,Person,Place,PostalAddress,กรุงเทพมหานคร,กรุงเทพและปริมณฑล,https://www.one2car.com/dealer/BoyAutogroup
7,ListItem,8,Product,Car,2019 Mitsubishi Triton 2.4 DOUBLE CAB (ปี 18-2...,[https://img1.icarcdn.com/29508811/thumb-l_%E0...,🔥b1025 mitsubishi all new triton double cab 2....,Brand,Mitsubishi,Person,...,619000,THB,https://schema.org/UsedCondition,https://schema.org/InStock,Person,Place,PostalAddress,กรุงเทพมหานคร,กรุงเทพและปริมณฑล,https://www.one2car.com/dealer/BoyAutogroup
8,ListItem,9,Product,Car,2013 Honda CR-V 2.4 (ปี 12-16) EL SUV,[https://img4.icarcdn.com/64240511/thumb-l_%E0...,🔥a1749 honda crv 2.4 el 2013 a/t ฟรีดาวน์ ผ่อน...,Brand,Honda,Person,...,549000,THB,https://schema.org/UsedCondition,https://schema.org/InStock,Person,Place,PostalAddress,กรุงเทพมหานคร,กรุงเทพและปริมณฑล,https://www.one2car.com/dealer/BoyAutogroup
9,ListItem,10,Product,Car,2017 Honda City 1.5 (ปี 14-18) V i-VTEC Sedan,[http://img.youtube.com/vi/sSNo2Zfek_A/maxresd...,"Honda city 1.5 v 2017 a/t a1541 ผ่อนเริ่ม 6,8x...",Brand,Honda,Person,...,414000,THB,https://schema.org/UsedCondition,https://sc

#### Define Function

In [115]:
def get_basic_info(content_list):
    basic_info = []
    for item in content_list:
        basic_info.append(item.find_all('a', attrs={'class': 'block text--muted'}))
    return basic_info

def get_names(basic_info):
    names = []
    for item in basic_info:
        for i in item:
            names.append(i.find_all("h3", attrs = {"class" : "weight--normal flush--bottom"})[0].text.strip())
    return names

def get_years(basic_info):
    years = []
    for item in basic_info:
        for i in item:
            years.append(i.find_all("h3", attrs = {"class" : "car-ad-year"})[0].text.strip())
    return years

def get_prices(basic_info):
    prices = []
    for item in basic_info:
        for i in item:
            prices.append(i.find_all("div", attrs = {"class" : "car-ad-price"})[0].string.replace(u'\xa0', u' ').strip())
    return prices

def get_motor(basic_info):
    tables = []
    motors = []
    mileages = []
    data = [motors, mileages]
    for item in basic_info:
        for i in item:
            tables.append(i.find_all("table", attrs = {"class" : "used-specs-table"})[0])
    for table in tables:
        motors.append(table.find("td", attrs={"class" : "car-ad-cc"}).string)
        mileages.append(table.find("td", attrs={"class" : "car-ad-km"}).string)
    return data


##### Connect url of Target website 

In [122]:
base_url = "https://www.one2car.com/%E0%B8%A3%E0%B8%96-%E0%B8%AA%E0%B8%B3%E0%B8%AB%E0%B8%A3%E0%B8%B1%E0%B8%9A-%E0%B8%82%E0%B8%B2%E0%B8%A2?page_number=1&page_size=26"
response = get(base_url, headers=headers)
html_soup = BeautifulSoup(response.content, 'html.parser')
content_list = html_soup.find_all('div', attrs={'class':'c-listings-mobile'})


In [123]:
content_list

[]

In [116]:
basic_info = get_basic_info(content_list)
names1 = get_names(basic_info)

In [117]:
basic_info

[[<a class="block text--muted" href="/รถ-สำหรับ-ขาย/honda/cr-v?page_size=26">
  <h3 class="weight--normal flush--bottom">
                              ตลาดรถ Honda CR-V มือสอง                        </h3>
  </a>],
 [<a class="block text--muted" href="/รถ-สำหรับ-ขาย/ford/everest?page_size=26">
  <h3 class="weight--normal flush--bottom">
                              ตลาดรถ Ford Everest มือสอง                        </h3>
  </a>],
 [<a class="block text--muted" href="/รถ-สำหรับ-ขาย/toyota/innova?page_size=26">
  <h3 class="weight--normal flush--bottom">
                              ตลาดรถ Toyota Innova มือสอง                        </h3>
  </a>],
 [<a class="block text--muted" href="/รถ-สำหรับ-ขาย/isuzu/d-max?page_size=26">
  <h3 class="weight--normal flush--bottom">
                              ตลาดรถ Isuzu D-Max มือสอง                        </h3>
  </a>],
 [<a class="block text--muted" href="/รถ-สำหรับ-ขาย/mercedes-benz/g-class/g400?page_size=26">
  <h3 class="weight--normal flush-

In [28]:
names1

['ตลาดรถ Volvo XC 40 มือสอง',
 'ตลาดรถ Honda Accord มือสอง',
 'ตลาดรถ Nissan Almera มือสอง',
 'ตลาดรถ Ford Ranger มือสอง',
 'ตลาดรถ Toyota C-HR มือสอง',
 'ตลาดรถ Ford Fiesta มือสอง',
 'ตลาดรถ Honda Jazz มือสอง',
 'ตลาดรถ Toyota Hilux Vigo มือสอง',
 'ตลาดรถ Mercedes-Benz E200 มือสอง',
 'ตลาดรถ Mercedes-Benz S300 มือสอง',
 'ตลาดรถ Suzuki Swift มือสอง',
 'ตลาดรถ Mazda CX-5 มือสอง',
 'รถตู้',
 'รถ 5 ประตู',
 'รถสปอร์ตมือสอง',
 'รถเก๋งมือสอง',
 'รถกระบะ',
 'รถ SUV',
 'ขายรถ Toyota Hilux Revo 2019',
 'ขายรถ Nissan March 2012',
 'ขายรถ Honda Jazz 2012',
 'ขายรถ Toyota Camry 2018',
 'ขายรถ Chevrolet Colorado 2017',
 'ขายรถ Toyota Hilux Revo 2017',
 'ขายรถ Honda Accord 2017',
 'ขายรถ Toyota Vios 2012',
 'ขายรถ BMW X1 2015',
 'ขายรถ Isuzu D-Max 2015',
 'ขายรถ BMW X6 2016',
 'ขายรถ Mitsubishi Triton 2019',
 'รถเปิดตัวใหม่',
 'มอเตอร์ไซค์เปิดตัวใหม่',
 'รีวิวรถยนต์',
 'รีวิวรถมอเตอร์ไซค์',
 'ราคารถยนต์และตารางผ่อน',
 'ราคามอเตอร์ไซค์และตารางผ่อน',
 'ข่าวรถหรู',
 'รีวิวรถ EV',
 'รถยนต์ไฟฟ้า EV']

In [61]:
print(html_soup.prettify())

<!DOCTYPE html>
<!--[if IE 8]><html lang="th" class="ie8"><![endif]-->
<!--[if !IE 8]><!-->
<html lang="th">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport">
   <title>
    ค้นหารถมือสอง ราคาถูก สภาพดี ในตลาดรถมือสองทั่วประเทศ | One2car
   </title>
   <!-- Global site tag (gtag.js) - Google Analytics -->
   <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-1389838-1">
   </script>
   <script>
    window.dataLayer = window.dataLayer || [];

    function gtag() {
        dataLayer.push(arguments);
    }

    gtag('js', new Date());

    gtag('config', 'UA-1389838-1', {'optimize_id': 'GTM-56539PK', 'send_page_view': false});
   </script>
   <meta content="_csrf" name="csrf-param"/>
   <meta content="xp_-R0tM9jKlbB47dzarcnFNcGOXNbGSElf0oZjX4cr_rapwf3ieQeYv

In [11]:
page_number = 1
names = []
prices = []
years = []
motors = []
mileages = []


for i in range(9):
    base_url = "https://www.one2car.com/%E0%B8%A3%E0%B8%96-%E0%B8%AA%E0%B8%B3%E0%B8%AB%E0%B8%A3%E0%B8%B1%E0%B8%9A-%E0%B8%82%E0%B8%B2%E0%B8%A2?page_number={}&page_size=26".format(page_number)
    response = get(base_url, headers=headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    content_list = html_soup.find_all('div', attrs={'class': 'car-ad sft-ad'})

    basic_info = get_basic_info(content_list)
    names1 = get_names(basic_info)
    prices1 = get_prices(basic_info)
    years1 = get_years(basic_info)
    motors1 = get_motor(basic_info)[0]
    mileages1 = get_motor(basic_info)[1]

    names.extend(names1)
    prices.extend(prices1)
    years.extend(years1)
    motors.extend(motors1)
    mileages.extend(mileages1)
    page_number = page_number + 1
    time.sleep(random.randint(1,2))

cols = ["Name", "Year", "Motor", "Mileage (Km)", "Price"]
data = pd.DataFrame({"Name" : names, "Year" : years, "Motor" : motors, "Mileage (Km)": mileages, "Price" : prices})[cols]
data["Price"] = data["Price"].replace({'\$ ':''}, regex = True)
data["Price"] = data["Price"].replace({'\,':''}, regex = True)
data["Mileage (Km)"] = data["Mileage (Km)"].replace({'\ Km':''}, regex = True)
data[["Mileage (Km)", "Year", "Motor", "Price"]] = data[["Mileage (Km)", "Year", "Motor", "Price"]].apply(pd.to_numeric)

data.head()
data.drop_duplicates().to_excel('Car_list.xls')
#Some filters
#data.loc[(data["Price"] < 50000000) & (data["Motor"] == 1200)].drop_duplicates()

KeyboardInterrupt: 